In [7]:
from urllib.request import urlretrieve
import zipfile
from pathlib import Path
import os

url = 'https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_POA_for_AUS_short-header.zip'

urlretrieve(url, Path('../data', 'pop_zip'))


(PosixPath('../data/pop_zip'), <http.client.HTTPMessage at 0x7fb1177b37f0>)

In [13]:
with zipfile.ZipFile('../data/pop_zip', 'r') as zip_ref:
    zip_ref.extractall('../data')

In [14]:
import shutil

shutil.rmtree('../data/Metadata')
shutil.rmtree('../data/Readme')

In [15]:
os.rename("../data/2021 Census GCP Postal Areas for AUS", "../data/census")

In [18]:
import pandas as pd
G01 = pd.read_csv("../data/census/2021Census_G01_AUST_POA.csv")
G02 = pd.read_csv("../data/census/2021Census_G02_AUST_POA.csv")

In [20]:
dataset = pd.merge(G01, G02, how="outer", on=["POA_CODE_2021", "POA_CODE_2021"])
dataset = dataset[["POA_CODE_2021", "Tot_P_M", "Tot_P_F","Tot_P_P", "High_yr_schl_comp_Yr_12_eq_M", "High_yr_schl_comp_Yr_12_eq_F", "High_yr_schl_comp_Yr_12_eq_P", "Median_age_persons", "Median_mortgage_repay_monthly", "Median_tot_prsnl_inc_weekly", "Median_tot_hhd_inc_weekly"]]

In [21]:
# Feature Engineering

import re

def strip_it(code):
    return code[3:]

dataset['POA_CODE_2021'] = dataset['POA_CODE_2021'].apply(strip_it)
dataset['comp_Yr_12_eq_percent'] = dataset['High_yr_schl_comp_Yr_12_eq_P'] / dataset['Tot_P_P']
dataset['comp_Yr_12_eq_percent_M'] = dataset['High_yr_schl_comp_Yr_12_eq_M'] / dataset['Tot_P_M']
dataset['comp_Yr_12_eq_percent_F'] = dataset['High_yr_schl_comp_Yr_12_eq_F'] / dataset['Tot_P_F']
dataset['house_repay_to_income'] = dataset["Median_mortgage_repay_monthly"] / (dataset["Median_tot_hhd_inc_weekly"] * 4.333333)


# "Median_age_persons", "Median_mortgage_repay_monthly", "Median_tot_prsnl_inc_weekly", "Median_tot_hhd_inc_weekly"

In [22]:
dataset2 = dataset[["POA_CODE_2021", "comp_Yr_12_eq_percent", "comp_Yr_12_eq_percent_M", "comp_Yr_12_eq_percent_F", "house_repay_to_income", "Median_age_persons", "Median_tot_prsnl_inc_weekly", "Median_mortgage_repay_monthly"]]
dataset2

,POA_CODE_2021,comp_Yr_12_eq_percent,comp_Yr_12_eq_percent_M,comp_Yr_12_eq_percent_F,house_repay_to_income,Median_age_persons,Median_tot_prsnl_inc_weekly,Median_mortgage_repay_monthly
0,2000,0.771728,0.769739,0.773937,0.290406,32,941,2800
1,2007,0.760864,0.760563,0.760977,0.319625,30,772,2500
2,2008,0.809423,0.801505,0.816840,0.343643,28,860,2600
3,2009,0.705720,0.709748,0.700973,0.266785,37,1297,2800
4,2010,0.756949,0.753972,0.761866,0.291350,36,1479,2900
...,...,...,...,...,...,...,...,...
2638,2899,0.388483,0.332075,0.442478,0.253378,50,736,1300
2639,6798,0.315012,0.268390,0.386861,0.142248,38,1233,1300
2640,6799,0.293423,0.251656,0.349315,0.180658,40,741,1972
2641,9494,0.283777,0.247478,0.328896,NaN,0,0,0


In [24]:
dataset2.to_csv('../data/curated/census.csv', header=True, index=False)